In [5]:
from scrapy import spiders

In [6]:
class ResearchgateSpider(spiders):
    name = 'researchgate'
    
    TYPE = "//span[@data-testid='research-meta-type']/text()" #DONE
    TITLE = "//h1/text()" #DONE
    ABSTRACT = "//div[@itemprop='description']/text()" #DONE
    
    AUTHORS = 'li.publication-author-list__item' #return array it should be wrapped authors.css('a::text).get
    DATE = "//div[@class='research-detail-header-section__metadata']/div/ul/li/text()"
'
    REFERENCES = ''
    
    def __init__(self, start_urls, maximum_no=2000):
        self.start_urls = start_urls
        self.crawled = start_urls.copy()
        self.maximum_no = maximum_no
    
    def start_requests(self):
        urls = self.start_urls
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        url = response.url
        id = url.split('/')[-1].split('_')[0]
        
        
# START_URLS = [
#     'https://www.researchgate.net/publication/328230984_BERT_Pre-training_of_Deep_Bidirectional_Transformers_for_Language_Understanding',
#     'https://www.researchgate.net/publication/323694313_The_Lottery_Ticket_Hypothesis_Training_Pruned_Neural_Networks/references',
#     'https://www.researchgate.net/publication/317558625_Attention_Is_All_You_Need'   
# ]

TypeError: module.__init__() takes at most 2 arguments (3 given)